
# EDA — Datathon Fase 5 (Decision)

Este notebook carrega as três bases do GitHub (`applicants.xlsx`, `prospects.xlsx`, `vagas.xlsx`), 
faz uma exploração inicial e aponta possíveis *join keys* (chaves de relacionamento).


In [ ]:

# %% [setup] Imports
import sys, os, re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Facilita importar os módulos locais se você rodar ao lado do repo
sys.path.append(os.path.abspath('../src'))
try:
    from preprocessing import normalize_applicants, normalize_prospects, normalize_vagas
except Exception as e:
    print("Observação: módulos de preprocessing ainda não disponíveis ou caminho não encontrado:", e)


In [ ]:

# %% [data] Caminhos (ajuste se precisar)
BASE = "https://raw.githubusercontent.com/rosimotasolucoes-byte/Conexao-certa/main/Dadoss"
URL_APPLICANTS = f"{BASE}/applicants.xlsx"
URL_PROSPECTS  = f"{BASE}/prospects.xlsx"
URL_VAGAS      = f"{BASE}/vagas.xlsx"

print(URL_APPLICANTS)
print(URL_PROSPECTS)
print(URL_VAGAS)


In [ ]:

# %% [load] Carregar dados
applicants = pd.read_excel(URL_APPLICANTS)
prospects  = pd.read_excel(URL_PROSPECTS)
vagas      = pd.read_excel(URL_VAGAS)

applicants_head = applicants.head(3)
prospects_head  = prospects.head(3)
vagas_head      = vagas.head(3)

applicants_shape = applicants.shape
prospects_shape  = prospects.shape
vagas_shape      = vagas.shape

applicants_shape, prospects_shape, vagas_shape


In [ ]:

# %% [preview] Mostrar nomes de colunas (minúsculas) e primeiras linhas
def preview(df, name):
    print(f"\n===== {name} =====")
    print("shape:", df.shape)
    print("columns:", list(df.columns))
    display(df.head(5))

preview(applicants, "applicants")
preview(prospects, "prospects")
preview(vagas, "vagas")


In [ ]:

# %% [clean] Normalizações básicas (se módulos estiverem disponíveis)
try:
    applicants_clean = normalize_applicants(applicants)
except:
    applicants_clean = applicants.copy()

try:
    prospects_clean = normalize_prospects(prospects)
except:
    prospects_clean = prospects.copy()

try:
    vagas_clean = normalize_vagas(vagas)
except:
    vagas_clean = vagas.copy()

preview(applicants_clean, "applicants_clean")
preview(prospects_clean, "prospects_clean")
preview(vagas_clean, "vagas_clean")


In [ ]:

# %% [missing] Percentual de valores ausentes por coluna
def missing_report(df):
    miss = df.isna().mean().sort_values(ascending=False)
    return miss.to_frame("missing_rate").query("missing_rate > 0")

mr_app = missing_report(applicants_clean)
mr_pro = missing_report(prospects_clean)
mr_vag = missing_report(vagas_clean)

print("\n--- Missing: applicants ---")
display(mr_app.head(20))
print("\n--- Missing: prospects ---")
display(mr_pro.head(20))
print("\n--- Missing: vagas ---")
display(mr_vag.head(20))


In [ ]:

# %% [keys] Buscar sugestões de chaves de relacionamento
def suggest_keys(df, prefix=None):
    cols = [c for c in df.columns if re.search(r'(id|vaga|job|position|cand|applicant)', str(c), re.I)]
    if prefix:
        cols = [c for c in cols if prefix.lower() in c.lower()] or cols
    return cols

cand_keys = suggest_keys(applicants_clean)
pros_keys = suggest_keys(prospects_clean)
vaga_keys = suggest_keys(vagas_clean)

print("Possíveis chaves em applicants:", cand_keys)
print("Possíveis chaves em prospects :", pros_keys)
print("Possíveis chaves em vagas     :", vaga_keys)

# Cruzamentos de candidatos e vagas dentro de prospects (heurística)
cand_like = [c for c in pros_keys if re.search(r'(cand|applicant)', c, re.I)]
vaga_like = [c for c in pros_keys if re.search(r'(vaga|job|position)', c, re.I)]
cand_like, vaga_like


In [ ]:

# %% [text] Medidas simples de texto (tamanho) nas tabelas
def text_len_stats(df, top=10):
    obj = df.select_dtypes(include='object')
    stats = {}
    for c in obj.columns:
        l = obj[c].fillna('').astype(str).str.len()
        stats[c] = {
            'non_null': int(l.gt(0).sum()),
            'mean_len': float(l.mean()),
            'p90_len': float(l.quantile(0.90))
        }
    return pd.DataFrame(stats).T.sort_values('non_null', ascending=False).head(top)

print("\n--- Comprimento de textos (applicants) ---")
display(text_len_stats(applicants_clean))
print("\n--- Comprimento de textos (vagas) ---")
display(text_len_stats(vagas_clean))


In [ ]:

# %% [plots] Exemplo de gráfico: top 10 colunas com mais preenchimento em applicants
obj = applicants_clean.select_dtypes(include='object')
fill_rate = obj.notna().mean().sort_values(ascending=False).head(10)
plt.figure()
fill_rate.plot(kind='bar', title='Top 10 colunas textuais mais preenchidas (applicants)')
plt.tight_layout()
plt.show()
